In [20]:
# 导入设计库
import svgwrite

In [64]:
# os库与操作系统交互
import os 
# time库提供与时间相关的功能
import time

In [21]:
# 根据真实尺度获取到设计尺度，
def Get_Designed_Size(Length,Width,High):
    return Length+15,Width+2,High+2

In [22]:
# 根据设计尺度得到绘制svg时使用的展示尺度，也是最后输出的需要裁剪的大块儿尺度
def Get_Show_Size(Length,Width,High):
    Show_Length = Length+(High+5+68.5)*2
    Show_High = Width*2+High*3+5*2
    return Show_Length,Show_High

In [23]:
# 获取折弯尺寸
def Get_Bend_Size(Length,Bend):
    Bend_Size = Length-Bend*2;
    return Bend_Size

In [24]:
# 获取线段一半的长度
def Get_MidLine(Length):
    return Length/2

In [25]:
# 获取设计尺度的中点，准备绘制
def Get_Show_Mid(Length):
    return Length/2

In [26]:
# 定义真实尺度
real_length = 300
real_width = 200
real_high = 60

In [27]:
# 首先传入真实尺度，得到设计尺度和展示尺度
designed_length,designed_width,designed_high = Get_Designed_Size(real_length,real_width,real_high)
show_length,show_high = Get_Show_Size(designed_length,designed_width,designed_high)

In [105]:
# 绘制最上方的折角
def Draw_UP(dwg):
    # 从展示尺度得到中点
    show_midpoint = Get_Show_Mid(show_length)
    # 从设计尺度得到一半长度
    designed_halflength = Get_MidLine(designed_length)
    # 得到最上方横线的起点和终点
    start_point = show_midpoint - designed_halflength
    stop_point = show_midpoint + designed_halflength
    # y轴嘴上方为0，从0开始绘制
    Horizontal = 0
    dwg.add(dwg.line((start_point,Horizontal), (stop_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折弯
    bendpoint_left = start_point - 2.5
    bendpoint_right = stop_point + 2.5
    dwg.add(dwg.line((bendpoint_left,Horizontal+2.5), (start_point,Horizontal), stroke='blue', stroke_width=2))
    dwg.add(dwg.line((bendpoint_right,Horizontal+2.5), (stop_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折角圆弧，起点为折弯点，终点为
    bendarc_left = bendpoint_left - 52
    bendarc_right = bendpoint_right + 52
    # 左右起点切线角度 水平向左或向右
    start_angle_left = math.radians(180)
    start_angle_right = 0
    # 起/终点切线长度
    start_tangent_length = 5
    stop_tangent_length = 5
    # 终点切线角度 (例如向上 45 度)
    end_angle = math.radians(90)
    # 终点切线长度
    end_tangent_length = 50
    
    # 定义起点和终点
    start_point = (bendpoint_left,Horizontal+2.5)
    end_point = (bendarc_left, Horizontal+2.5+45.5)
    
    # 计算左侧控制点
    start_control_x = start_point[0] + start_tangent_length * math.cos(start_angle)
    start_control_y = start_point[1] + start_tangent_length * math.sin(start_angle)

    end_control_x = end_point[0] - end_tangent_length * math.cos(end_angle)
    end_control_y = end_point[1] - end_tangent_length * math.sin(end_angle)
    # 创建路径
    path = dwg.path(
    d=[
        'M', start_point,
        'C', (start_control_x, start_control_y), (end_control_x, end_control_y), end_point
    ],
    fill='none',
    stroke='blue',
    stroke_width=2
    )
    dwg.add(path)
    
    # 尝试镜像
    # 复制原始路径
    path_mirrored = path.copy()

    # 计算镜像的中心点 (假设对称轴是垂直的，位于 x=50)
    mirror_x = show_midpoint
    translation_x = 2 * mirror_x  # 需要平移以正确对齐
    
    # 应用水平镜像和位移变换
    path_mirrored.translate(tx=translation_x)
    path_mirrored.scale(sx=-1, sy=1)
    dwg.add(path_mirrored)
    # # 可视化起点和终点
    # dwg.add(dwg.circle(start_point, 5, fill='red'))
    # dwg.add(dwg.circle(end_point, 5, fill='green'))
    
    # # 可视化控制点 (仅用于理解)
    # dwg.add(dwg.circle((start_control_x, start_control_y), 3, fill='orange'))
    # dwg.add(dwg.circle((end_control_x, end_control_y), 3, fill='purple'))
    # dwg.add(dwg.line(start_point, (start_control_x, start_control_y), stroke='gray', stroke_dasharray="5,5"))
    # dwg.add(dwg.line(end_point, (end_control_x, end_control_y), stroke='gray', stroke_dasharray="5,5"))

In [151]:
# 绘制包装盒函数，使用镜像的方式，只绘制左侧一半，然后镜像
def Drwa_All(dwg):
    # 从展示尺度得到中点
    show_midpoint = Get_Show_Mid(show_length)
    # 从设计尺度得到一半长度
    designed_halflength = Get_MidLine(designed_length)
    # 得到最上方横线的起点和终点
    start_point = show_midpoint - designed_halflength
    stop_point = show_midpoint
    # y轴嘴上方为0，从0开始绘制
    Horizontal = 0
    dwg.add(dwg.line((start_point,Horizontal), (stop_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折弯
    bendpoint_left = start_point - 2.5
    dwg.add(dwg.line((bendpoint_left,Horizontal+2.5), (start_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折角圆弧，起点为折弯点，终点为
    bendarc_left = bendpoint_left - 52
    # 左起点切线角度 水平向左
    start_angle_left = math.radians(180)
    # 起/终点切线长度
    start_tangent_length = 5
    stop_tangent_length = 5
    # 终点切线角度 (例如向上 45 度)
    end_angle = math.radians(90)
    # 终点切线长度
    
    # 定义起点和终点
    start_point = (bendpoint_left,Horizontal+2.5)
    end_point = (bendarc_left, Horizontal+2.5+45.5)
    
    # 计算左侧控制点
    start_control_x = start_point[0] + start_tangent_length * math.cos(start_angle)
    start_control_y = start_point[1] + start_tangent_length * math.sin(start_angle)

    end_control_x = end_point[0] - end_tangent_length * math.cos(end_angle)
    end_control_y = end_point[1] - end_tangent_length * math.sin(end_angle)
    # 创建路径
    path = dwg.path(
    d=[
        'M', start_point,
        'C', (start_control_x, start_control_y), (end_control_x, end_control_y), end_point
    ],
    fill='none',
    stroke='blue',
    stroke_width=2
    )
    dwg.add(path)
    # 开始绘制圆弧
    # 定义圆弧的圆心 
    Cx = bendarc_left+13
    Cy = Horizontal+2.5+45.5
    # 定义圆弧起点
    C_startpoint_x = bendarc_left
    C_startpoint_y = Cy
    # 定于圆弧终点
    C_endpoint_x = Cx
    C_endpoint_y = Cy+13
    # 已知半径和扫过角度
    radius = 13
    sweep_angle_degrees = 90
    # 确定 large-arc-flag
    large_arc = 0 if abs(sweep_angle_degrees) <= 180 else 1
    # 确定 sweep-flag (需要根据起点和终点相对于圆心的位置来判断方向)
    # 这里需要你根据期望的弧形走向来设置 0 (逆时针) 或 1 (顺时针)
    # 由于终点 y 坐标小于起点 y 坐标，且终点 x 坐标大于起点 x 坐标，
    # 如果我们期望一个向下弯曲的弧，顺时针可能是正确的。
    sweep = 0
    # 构建路径
    path_d = [
        'M', (C_startpoint_x, C_startpoint_y),
        'A', (radius, radius), 0, large_arc, sweep, (C_endpoint_x, C_endpoint_y)
    ]
    
    # 创建路径元素并添加到 Drawing 对象
    path = dwg.path(d=path_d, fill='none', stroke='blue', stroke_width=2)
    dwg.add(path)
    # 绘制直线
    dwg.add(dwg.line((C_endpoint_x,C_endpoint_y), (C_endpoint_x+41.5,C_endpoint_y), stroke='blue', stroke_width=2))
    # 绘制圆弧
    # 定义圆弧的圆心 
    Cx = C_endpoint_x+41.5
    Cy = C_endpoint_y+1
    # 定义圆弧起点
    C_startpoint_x = Cx
    C_startpoint_y = Cy-1
    # 定于圆弧终点
    C_endpoint_x = Cx+1
    C_endpoint_y = Cy
    # 已知半径和扫过角度
    radius = 11
    sweep_angle_degrees = 90
    # 确定 large-arc-flag
    large_arc = 0 if abs(sweep_angle_degrees) <= 180 else 1
    sweep = 1
    # 构建路径
    path_d = [
        'M', (C_startpoint_x, C_startpoint_y),
        'A', (radius, radius), 0, large_arc, sweep, (C_endpoint_x, C_endpoint_y)
    ]
    
    # 创建路径元素并添加到 Drawing 对象
    path = dwg.path(d=path_d, fill='none', stroke='blue', stroke_width=2)
    dwg.add(path)
    # 绘制圆弧
    # 已知半径和扫过角度
    radius = 1.5
    sweep_angle_degrees = 70
    sweep_angle_radians = math.radians(sweep_angle_degrees)
    # 定义圆弧的圆心 
    Cx = C_endpoint_x - 1.5
    Cy = C_endpoint_y
    # 定义圆弧起点
    C_startpoint_x = C_endpoint_x
    C_startpoint_y = Cy
    # 定于圆弧终点
    start_angle_radians = math.atan2(C_startpoint_y - Cy, C_startpoint_x - Cx)
    end_angle_radians = start_angle_radians + sweep_angle_radians
    C_endpoint_x = Cx + radius * math.cos(end_angle_radians)
    C_endpoint_y = Cy + radius * math.sin(end_angle_radians)

    # 确定 large-arc-flag
    large_arc = 0 if abs(sweep_angle_degrees) <= 180 else 1
    sweep = 1
    # 构建路径
    path_d = [
        'M', (C_startpoint_x, C_startpoint_y),
        'A', (radius, radius), 0, large_arc, sweep, (C_endpoint_x, C_endpoint_y)
    ]
    
    # 创建路径元素并添加到 Drawing 对象
    path = dwg.path(d=path_d, fill='none', stroke='blue', stroke_width=2)
    dwg.add(path)

In [128]:
# 绘制包装盒函数，使用镜像的方式，只绘制左侧一半，然后镜像
def Draw_UP(dwg):
    # 从展示尺度得到中点
    show_midpoint = Get_Show_Mid(show_length)
    # 从设计尺度得到一半长度
    designed_halflength = Get_MidLine(designed_length)
    # 得到最上方横线的起点和终点
    start_point = show_midpoint - designed_halflength
    stop_point = show_midpoint + designed_halflength
    # y轴嘴上方为0，从0开始绘制
    Horizontal = 0
    dwg.add(dwg.line((start_point,Horizontal), (stop_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折弯
    bendpoint_left = start_point - 2.5
    bendpoint_right = stop_point + 2.5
    dwg.add(dwg.line((bendpoint_left,Horizontal+2.5), (start_point,Horizontal), stroke='blue', stroke_width=2))
    dwg.add(dwg.line((bendpoint_right,Horizontal+2.5), (stop_point,Horizontal), stroke='blue', stroke_width=2))
    # 绘制折角圆弧，起点为折弯点，终点为
    bendarc_left = bendpoint_left - 52
    bendarc_right = bendpoint_right + 52
    # 左右起点切线角度 水平向左或向右
    start_angle_left = math.radians(180)
    start_angle_right = 0
    # 起/终点切线长度
    start_tangent_length = 5
    stop_tangent_length = 5
    # 终点切线角度 (例如向上 45 度)
    end_angle = math.radians(90)
    
    # 定义起点和终点
    start_point = (bendpoint_left,Horizontal+2.5)
    end_point = (bendarc_left, Horizontal+2.5+45.5)
    
    # 计算左侧控制点
    start_control_x = start_point[0] + start_tangent_length * math.cos(start_angle)
    start_control_y = start_point[1] + start_tangent_length * math.sin(start_angle)

    end_control_x = end_point[0] - end_tangent_length * math.cos(end_angle)
    end_control_y = end_point[1] - end_tangent_length * math.sin(end_angle)
    # 创建路径
    path = dwg.path(
    d=[
        'M', start_point,
        'C', (start_control_x, start_control_y), (end_control_x, end_control_y), end_point
    ],
    fill='none',
    stroke='blue',
    stroke_width=2
    )
    dwg.add(path)
    
   
    # # 可视化起点和终点
    # dwg.add(dwg.circle(start_point, 5, fill='red'))
    # dwg.add(dwg.circle(end_point, 5, fill='green'))
    
    # # 可视化控制点 (仅用于理解)
    # dwg.add(dwg.circle((start_control_x, start_control_y), 3, fill='orange'))
    # dwg.add(dwg.circle((end_control_x, end_control_y), 3, fill='purple'))
    # dwg.add(dwg.line(start_point, (start_control_x, start_control_y), stroke='gray', stroke_dasharray="5,5"))
    # dwg.add(dwg.line(end_point, (end_control_x, end_control_y), stroke='gray', stroke_dasharray="5,5"))

In [152]:
# 创建一幅svg图像，命名格式为drawing+时间戳
def Create_SVG():
    # 通过展示尺度创建一张svg图像
    # 获取当前时间戳
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    base_filename="drawing"
    filename = f"{base_filename}_{timestamp}.svg"
    dwg = svgwrite.Drawing(filename, size=(show_length,show_high))
    # Draw_UP(dwg)
    Drwa_All(dwg)
    dwg.save()

In [153]:
Create_SVG()

In [15]:
dwg = svgwrite.Drawing('my_test_drawing_new.svg', size=(show_length,show_high))

315

In [73]:
timestamp

'20250515_153309'

In [75]:
filename

'drawing_20250515_153421.svg'

In [87]:
import svgwrite
import math

dwg = svgwrite.Drawing('tangent_arc.svg', profile='tiny')

# 起点和终点
start_point = (50, 50)
end_point = (200, 100)

# 起点切线角度 (例如水平向右)
start_angle = 0
# 起点切线长度
start_tangent_length = 50

# 终点切线角度 (例如向上 45 度)
end_angle = math.radians(45)
# 终点切线长度
end_tangent_length = 50

# 计算控制点
start_control_x = start_point[0] + start_tangent_length * math.cos(start_angle)
start_control_y = start_point[1] + start_tangent_length * math.sin(start_angle)

end_control_x = end_point[0] - end_tangent_length * math.cos(end_angle)
end_control_y = end_point[1] - end_tangent_length * math.sin(end_angle)

# 创建路径
path = dwg.path(
    d=[
        'M', start_point,
        'C', (start_control_x, start_control_y), (end_control_x, end_control_y), end_point
    ],
    fill='none',
    stroke='blue',
    stroke_width=2
)
dwg.add(path)

# 可视化起点和终点
dwg.add(dwg.circle(start_point, 5, fill='red'))
dwg.add(dwg.circle(end_point, 5, fill='green'))

# 可视化控制点 (仅用于理解)
dwg.add(dwg.circle((start_control_x, start_control_y), 3, fill='orange'))
dwg.add(dwg.circle((end_control_x, end_control_y), 3, fill='purple'))
dwg.add(dwg.line(start_point, (start_control_x, start_control_y), stroke='gray', stroke_dasharray="5,5"))
dwg.add(dwg.line(end_point, (end_control_x, end_control_y), stroke='gray', stroke_dasharray="5,5"))

dwg.save()

In [104]:
import svgwrite

dwg = svgwrite.Drawing('symmetric_shape.svg', profile='tiny')

# 绘制原始一半的图形 (例如一个简单的三角形)
path_original = dwg.path(d=['M', (50, 50), 'L', (100, 75), 'L', (75, 120), 'Z'],
                            fill='blue', stroke='black')
dwg.add(path_original)

# 复制原始路径
path_mirrored = path_original.copy()
path_mirrored.fill = 'red'

# 计算镜像的中心点 (假设对称轴是垂直的，位于 x=50)
mirror_x = 50
translation_x = 2 * mirror_x  # 需要平移以正确对齐

# 应用水平镜像和位移变换
path_mirrored.translate(tx=translation_x)
path_mirrored.scale(sx=-1, sy=1)
dwg.add(path_mirrored)

dwg.save()